In [6]:
import boto3
import os
import json

In [7]:
def setup_environment():
    import sys
    sys.path.append('C:\\gitworkspace\\aimldemo\\jupyterworkapce')
    import stratup_env_setup
    stratup_env_setup.set_env()

In [8]:
setup_environment()

In [9]:
 s3_client = boto3.client('s3')

In [10]:
response = s3_client.list_buckets()

In [11]:
    if 'Buckets' in response:
        print("S3 Buckets in your account:")
        for bucket in response['Buckets']:
            print(f"- {bucket['Name']}")
    else:
        print("No buckets found.")

S3 Buckets in your account:
- aws-cloudtrail-logs-312296519090-783817be
- aws-emr-resources-312296519090-us-east-1
- aws-logs-312296519090-us-east-1
- elasticbeanstalk-us-east-1-312296519090
- elasticbeanstalk-us-east-2-312296519090
- sagemaker-studio-o5xeslsirg
- sagemaker-us-east-1-312296519090
- test-s3-jj-bucket


# Amazon Titan Model

In [ ]:
bedrock = boto3.client(region_name="us-east-1", service_name='bedrock')

In [ ]:
bedrock.list_foundation_models()

In [ ]:
bedrock.get_foundation_model(modelIdentifier='amazon.titan-text-express-v1')

In [ ]:
bedrock_runtime = boto3.client(region_name="us-east-1",
                               service_name='bedrock-runtime')

Thus, the Titan model expects the following jsonified request body:
```
{
    "inputText": string,
    "textGenerationConfig": {
        "temperature": float,  
        "topP": float,
        "maxTokenCount": int,
        "stopSequences": [string]
    }
}
```

In [ ]:
question = "What is the capital of New Jersey?"

In [ ]:
body = json.dumps({
    "inputText": question,
})
print(body)

In [ ]:
response = bedrock_runtime.invoke_model(body=body, modelId="amazon.titan-text-lite-v1")

In [ ]:
response_body = json.loads(response.get('body').read())
response_body

# LLama 2

Llama 2 accept the following parameters:

- `prompt`: string
  - This represents the initial text or question provided to the system.
- `temperature`: float
  - This controls the randomness in the text generation process. A higher temperature results in more random outputs, while a lower temperature produces more predictable text.
- `top_p`: float
  - This parameter, also known as "nucleus sampling," controls the diversity of the generated text. It sets a threshold to include the most likely next words, cumulatively adding up to the specified probability 'P'. A lower value ignores less probable options. Set to 0 or 1.0 to disable
- `max_gen_len`: int
  - This specifies the maximum number of tokens that the generated text can contain.
   
Thus, the model expects the following jsonified request body:
```
{
    "prompt": "string",
    "temperature": float,
    "top_p": float,
    "max_gen_len": int
}
```

Default Values:


| Category              | Parameter            | JSON object format | Minimum | Maximum | Default |
|-----------------------|----------------------|--------------------|---------|---------|---------|
| Randomness and diversity | Temperature        | `temperature`      | 0       | 1       | 0.5     |
|                       | Top P               | `top_p`            | 0       | 1       | 0.9     |
| Length                | Max generation length| `max_gen_len`      | 1       | 2048    | 512     |

In [ ]:
prompt = "Explain about Piscataway, NJ"

In [ ]:
body = json.dumps({
    "prompt": prompt,
    "max_gen_len": 256,
    "temperature": 0.9,  # Let's push it to be creative
    "top_p": 0.9,
})


In [ ]:
response = bedrock_runtime.invoke_model(body=body, modelId="meta.llama3-8b-instruct-v1:0")
response_body = json.loads(response.get('body').read())
response_body

# PDF RAG

In [ ]:
!pip install PyPDF2

In [ ]:
file_name = 'Corporate_Travel_Policy.pdf'

In [ ]:
from PyPDF2 import PdfReader

In [ ]:
reader = PdfReader(file_name)

In [ ]:
page = reader.pages[0]

In [ ]:
text = page.extract_text().replace('\n',' ')

In [ ]:
text

In [ ]:
query = "How many working days of PTO do employees get per year?"

In [ ]:
prompt = f"Answer the following question: {query}. Here is the reference text:\n{text}"

In [ ]:
body = json.dumps({
    "inputText": prompt,
    "textGenerationConfig": {
        "temperature": 0,
        "maxTokenCount": 4096
    }
})

In [ ]:
response = bedrock_runtime.invoke_model(body=body, modelId="amazon.titan-text-lite-v1")
response_body = json.loads(response.get('body').read())
print(response_body["results"][0]["outputText"])